In [11]:
import numpy as np
import matplotlib
import mellon
import matplotlib.pyplot as plt
from sklearn.cluster import k_means
import palantir
import scanpy as sc
import pandas as pd

In [12]:
!mkdir plots # create the directory  

# Mellon estimate for all cells

In [ ]:
"""
:::NOTE:::
This section requires retreiving the seurat .rds file and converting it to h5ad
After the GEO data is made public, this will be updated with code to convert GEO serurat object to h5ad.
Until then, the seurat object from GSE296507 will need to be manually retreived and converted
"""
h5ad_file="<manually_converted_seurat_object>"

# loading value
ad1 = sc.read_h5ad(h5ad_file)
# run diffusion map for density estimate
dm_res = palantir.utils.run_diffusion_maps(ad1, pca_key="X_pca", n_components=20)
# run mellon to estimate cell density
model = mellon.DensityEstimator()
log_density = model.fit_predict(ad1.obsm["DM_EigenVectors"])
predictor = model.predict
# add estimate value to the anndata object
ad1.obs["mellon_log_density"] = log_density
# To make the plot more clear, clip the low density range
ad1.obs["mellon_log_density_clipped"] = np.clip(
    log_density, *np.quantile(log_density, [0.05, 1])
)
# plot density map
sc.pl.scatter(
    ad1, color=["mellon_log_density", "mellon_log_density_clipped"], basis="umap"
)

# plot density plot for all cells at t0

In [14]:
# select the t0 data
ad_t0 = ad1[ad1.obs['scaled_time'] == 0]
sc.settings.set_figure_params(dpi_save=1200) # set high dpi


with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.scatter(
    ad_t0, color=["mellon_log_density", "mellon_log_density_clipped"], basis="umap", show=False
)
    plt.savefig("./plots/Mellon_all_t0.png") # save the plot

# plot density plot for all cells at t1

In [15]:
# select the t1 data
ad_t1 = ad1[ad1.obs['scaled_time'] == 1]
with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.scatter(
    ad_t1, color=["mellon_log_density", "mellon_log_density_clipped"], basis="umap", show=False
)
    plt.savefig("./plots/Mellon_all_t1.png") # save the plot

# plot density plot for the BC cells 

In [16]:
# select all the bc data
bc_data = ad1[ad1.obs['treatment'] == "CML_KO"]
sc.settings.set_figure_params(dpi_save=1200)


with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.scatter(
    bc_data, color=["mellon_log_density", "mellon_log_density_clipped"], basis="umap", show=False
)
    plt.savefig("./plots/Mellon_bc_all.png")

In [17]:
# select the bc data at t0
bc_data_t0 = bc_data[bc_data.obs['scaled_time'] == 0]
sc.settings.set_figure_params(dpi_save=1200)


with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.scatter(
    bc_data_t0, color=["mellon_log_density", "mellon_log_density_clipped"], basis="umap", show=False
)
    plt.savefig("./plots/Mellon_bc_t0.png")

In [18]:
# select the bc data at t1
bc_data_t1 = bc_data[bc_data.obs['scaled_time'] == 1]
sc.settings.set_figure_params(dpi_save=1200)


with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.scatter(
    bc_data_t1, color=["mellon_log_density", "mellon_log_density_clipped"], basis="umap", show=False
)
    plt.savefig("./plots/Mellon_bc_t1.png")